In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

2023-07-16 11:59:54.675386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 11:59:54.731712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 11:59:54.732891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 11:59:56.765778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../gait_standardised.csv')
df.head()

,subject,condition,replication,leg,joint,time,angle,angle_scaled
0,1,1,1,1,1,0,4.682881,-0.465902
1,1,1,1,1,1,1,5.073127,-0.441551
2,1,1,1,1,1,2,5.229774,-0.431776
3,1,1,1,1,1,3,5.083273,-0.440918
4,1,1,1,1,1,4,4.652399,-0.467804


## Coverting to correct data types


In [3]:
print(f"Data types before:\n{df.dtypes}\n")
categorical_columns = ['subject', 'condition', 'replication', 'leg', 'joint', 'time']
df[categorical_columns] = df[categorical_columns].astype('category')
df['time'] = df['time'].cat.set_categories(list(range(0, 101)), ordered=True)
print(f"Data types after:\n{df.dtypes}")

Data types before:
subject           int64
condition         int64
replication       int64
leg               int64
joint             int64
time              int64
angle           float64
angle_scaled    float64
dtype: object

Data types after:
subject         category
condition       category
replication     category
leg             category
joint           category
time            category
angle            float64
angle_scaled     float64
dtype: object


## Separate features (x) and target (y)

In [4]:
X = df.drop(columns=['angle', 'angle_scaled'])
y = df['angle_scaled']

## Implement Neural Network Model

In [5]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
# Initialize lists to store MSE scores from each fold
mse_scores = []

In [6]:
# Perform K-fold cross-validation
for train_idx, test_idx in kfold.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Create and compile the neural network model
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)

    # Calculate the mean squared error (MSE) and store it in the list
    mse_scores.append(mean_squared_error(y_test, y_pred))

# Calculate the average MSE across all folds
average_mse = np.mean(mse_scores)
print("Average MSE:", average_mse)


2023-07-16 12:00:01.213176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-16 12:00:01.393030: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


569/569 [==============================] - 1s 931us/step
Average MSE: 0.033507532154146175


In [8]:
df_MSE = pd.DataFrame({'Fold': range(1, len(mse_scores)+1), 'MSE': mse_scores})
print(df_MSE)

   Fold       MSE
0     1  0.034674
1     2  0.033323
2     3  0.041058
3     4  0.030483
4     5  0.029181
5     6  0.027451
6     7  0.036381
7     8  0.025643
8     9  0.038492
9    10  0.038389
